# Group 6 Lab 2 Task 3

## Group Members
* **127820** - Catherine Nduta
* **130816** - Natalie Ndetei
* **134205** - Ryan Muema
* **119199** - Mutiku Adina
* **127690** - Sharon Mwangi
* **134583** - Jeffrey Ongicho
* **136667** - Benard Wanyande

# Task 3
- Write a program to construct the LL (1) table for a given CFG. [The CFG can be static i.e. the same CFG all time or dynamic, meaning based on a given structure of your choice, one can specify any CFG and your program produces the relevant LL (1) table].

*Python is used below*

In [5]:
!pip install tabulate


In [11]:
from tabulate import tabulate

class LL1Parser:
    def __init__(self):
        self.grammar = {}  # Dictionary to store production rules
        self.non_terminals = set()
        self.terminals = set()
        self.first_sets = {}  # First sets for non-terminals
        self.follow_sets = {}  # Follow sets for non-terminals
        self.parsing_table = {}

    def add_production(self, non_terminal, production):
        if non_terminal not in self.grammar:
            self.grammar[non_terminal] = []
        self.grammar[non_terminal].append(production)

    def compute_first_sets(self):
        # Initialize First sets
        for non_terminal in self.grammar:
            self.first_sets[non_terminal] = set()

        while True:
            updated = False
            for non_terminal, productions in self.grammar.items():
                for production in productions:
                    for symbol in production:
                        if symbol in self.terminals:
                            if symbol not in self.first_sets[non_terminal]:
                                updated = True
                                self.first_sets[non_terminal].add(symbol)
                                break
                            else:
                                break
                        elif symbol in self.non_terminals:
                            if not self.first_sets[symbol].issubset(self.first_sets[non_terminal]):
                                updated = True
                                self.first_sets[non_terminal].update(self.first_sets[symbol])
                            if 'ε' not in self.first_sets[symbol]:
                                break
                    else:
                        if 'ε' not in self.first_sets[non_terminal]:
                            updated = True
                            self.first_sets[non_terminal].add('ε')
            if not updated:
                break

    def compute_follow_sets(self):
        # Initialize Follow sets
        for non_terminal in self.grammar:
            self.follow_sets[non_terminal] = set()

        start_symbol = next(iter(self.grammar))
        self.follow_sets[start_symbol].add('$')

        while True:
            updated = False
            for non_terminal, productions in self.grammar.items():
                for production in productions:
                    for i, symbol in enumerate(production):
                        if symbol in self.non_terminals:
                            rest = production[i + 1:]
                            if not rest:
                                if not self.follow_sets[symbol].issubset(self.follow_sets[non_terminal]):
                                    updated = True
                                    self.follow_sets[non_terminal].update(self.follow_sets[symbol])
                            else:
                                first_rest = self.compute_first(rest)
                                if 'ε' in first_rest:
                                    first_rest.remove('ε')
                                    if not first_rest.issubset(self.follow_sets[symbol]):
                                        updated = True
                                        self.follow_sets[symbol].update(first_rest)
                                        first_rest.add('ε')
                                        if not first_rest.issubset(self.follow_sets[non_terminal]):
                                            updated = True
                                            self.follow_sets[non_terminal].update(first_rest)
                                else:
                                    if not first_rest.issubset(self.follow_sets[symbol]):
                                        updated = True
                                        self.follow_sets[symbol].update(first_rest)
            if not updated:
                break

    def compute_first(self, symbols):
        first_set = set()
        for symbol in symbols:
            if symbol in self.terminals:
                first_set.add(symbol)
                break
            elif symbol in self.non_terminals:
                first_set.update(self.first_sets[symbol])
                if 'ε' not in self.first_sets[symbol]:
                    break
        else:
            first_set.add('ε')
        return first_set

    def build_parsing_table(self):
        for non_terminal, productions in self.grammar.items():
            for production in productions:
                first_set = self.compute_first(production)
                for symbol in first_set:
                    if symbol != 'ε':
                        self.parsing_table[(non_terminal, symbol)] = production

                if 'ε' in first_set:
                    follow_set = self.follow_sets[non_terminal]
                    for symbol in follow_set:
                        self.parsing_table[(non_terminal, symbol)] = production

    def print_parsing_table(self):
        headers = ["Non-terminal"] + sorted(self.terminals) + ['$']
        table = []

        for non_terminal in self.grammar:
            row = [non_terminal]
            for col in headers[1:]:
                if (non_terminal, col) in self.parsing_table:
                    production = self.parsing_table[(non_terminal, col)]
                    row.append(" -> ".join(production))
                else:
                    row.append("")
            table.append(row)

        table_str = tabulate(table, headers, tablefmt="fancy_grid")
        print("LL(1) Parsing Table:")
        print(table_str)

    def parse(self, input_string):
        stack = ['$']
        input_string += '$'
        input_index = 0
        production_sequence = []

        stack.append(next(iter(self.grammar)))  # Push the start symbol
        while stack:
            top_symbol = stack[-1]
            current_input = input_string[input_index]

            if top_symbol == current_input == '$':
                print("Accepted")
                break

            if top_symbol in self.terminals:
                if top_symbol == current_input:
                    stack.pop()
                    input_index += 1
                else:
                    print("Rejected")
                    break
            elif (top_symbol, current_input) in self.parsing_table:
                production = self.parsing_table[(top_symbol, current_input)]
                stack.pop()
                if production != 'ε':
                    stack.extend(reversed(production))
                production_sequence.append(production)
            else:
                print("Rejected")
                break

        print("Production Sequence:")
        for production in production_sequence:
            print(" -> ".join(production))


if __name__ == "__main__":
    parser = LL1Parser()

    # Define your grammar here (example: a simple arithmetic expression grammar)
    parser.non_terminals = {'E', 'T', 'F'}
    parser.terminals = {'+', '*', '(', ')', 'id'}

    parser.add_production('E', ['T', 'E\''])
    parser.add_production('E\'', ['+', 'T', 'E\''])
    parser.add_production('E\'', ['ε'])
    parser.add_production('T', ['F', 'T\''])
    parser.add_production('T\'', ['*', 'F', 'T\''])
    parser.add_production('T\'', ['ε'])
    parser.add_production('F', ['(', 'E', ')'])
    parser.add_production('F', ['id'])

    parser.compute_first_sets()
    parser.compute_follow_sets()
    parser.build_parsing_table()
    parser.print_parsing_table()

    input_string = "id + id * id"
    print("\nParsing Input:", input_string)
    parser.parse(input_string)


LL(1) Parsing Table:
╒════════════════╤═════════════╤═════╤══════════════╤══════════════╤═════════╤═════╕
│ Non-terminal   │ (           │ )   │ *            │ +            │ id      │ $   │
╞════════════════╪═════════════╪═════╪══════════════╪══════════════╪═════════╪═════╡
│ E              │ T -> E'     │     │              │              │ T -> E' │     │
├────────────────┼─────────────┼─────┼──────────────┼──────────────┼─────────┼─────┤
│ E'             │             │     │              │ + -> T -> E' │         │     │
├────────────────┼─────────────┼─────┼──────────────┼──────────────┼─────────┼─────┤
│ T              │ F -> T'     │     │              │              │ F -> T' │     │
├────────────────┼─────────────┼─────┼──────────────┼──────────────┼─────────┼─────┤
│ T'             │             │     │ * -> F -> T' │              │         │     │
├────────────────┼─────────────┼─────┼──────────────┼──────────────┼─────────┼─────┤
│ F              │ ( -> E -> ) │     │      

## Explanation (Logic behind the code provided)

The provided Python code constructs the LL(1) parsing table for a given context-free grammar (CFG) and provides the functionality to parse input strings based on the generated LL(1) parsing table. Let's break down the code and explain how it addresses the question of constructing the LL(1) table for a given CFG:

1. **Class `LL1Parser`**: This class encapsulates the logic for LL(1) parsing and table construction. It has several methods to perform various tasks related to LL(1) parsing and table generation.

2. **Initialization**:
   - The class initializes with empty data structures to store the CFG, non-terminals, terminals, first sets, follow sets, and the LL(1) parsing table.

3. **`add_production` Method**:
   - This method is used to add production rules to the grammar. It takes a non-terminal and a production as arguments and stores them in the `grammar` dictionary.

4. **`compute_first_sets` Method**:
   - It calculates the first sets for each non-terminal in the CFG using a while loop. It iteratively updates the first sets until they no longer change.
   - The first set of a non-terminal contains the terminal symbols that can appear as the first symbol of a string derived from that non-terminal.

5. **`compute_follow_sets` Method**:
   - This method calculates the follow sets for each non-terminal in the CFG. Follow sets represent the terminals that can immediately follow a non-terminal in any valid derivation.
   - It also uses a while loop to iteratively update the follow sets until they no longer change.

6. **`compute_first` Method**:
   - It calculates the first set of a given list of symbols. It is used during parsing to determine the first set of a sequence of symbols.

7. **`build_parsing_table` Method**:
   - This method constructs the LL(1) parsing table based on the computed first sets, follow sets, and grammar.
   - For each production in the grammar, it calculates the first set of the production and updates the parsing table with entries for the non-terminal and each symbol in its first set.
   - If the first set contains 'ε' (epsilon), it also updates the parsing table with entries for the follow set of the non-terminal.

8. **`print_parsing_table` Method**:
   - It prints the LL(1) parsing table in a tabulated format using the `tabulate` library, making it visually clear and organized.

9. **`parse` Method**:
   - This method performs LL(1) parsing on an input string using the constructed parsing table.
   - It maintains a stack to simulate the parsing process and a production sequence to track the applied productions.
   - The parsing process continues until the stack is empty and the input string is fully processed.

10. **Main Section**:
    - In the main section, an instance of the `LL1Parser` class is created.
    - A sample CFG (arithmetic expression grammar) is defined with non-terminals and terminals, and production rules are added.
    - The program computes the first sets, follow sets, and constructs the LL(1) parsing table.
    - It then allows the user to enter input strings and performs LL(1) parsing on each input string, displaying the result.

This code addresses the question by providing a reusable LL(1) parsing table generator for a given CFG. It calculates the necessary sets (first sets and follow sets) and builds a parsing table that allows parsing of input strings based on the provided grammar.